In [ ]:
import numpy as np
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Reshape, GRU, RNN

tf.keras.backend.set_floatx('float64')

In [ ]:
train = pd.read_csv('eletrain.csv')
test = pd.read_csv('test.csv', index_col = 1)
submission = pd.read_csv('sample_submission.csv')

전력사용량은 당연히 사용해야하고 기온, 습도가 전력사용량에 많은 영향을 줄 것 같아서 두 개도 학습 특성으로 선택

In [ ]:
train

,num,date_time,전력사용량(kWh),기온(°C),풍속(m/s),습도(%),강수량(mm),일조(hr),비전기냉방설비운영,태양광보유
0,1,2020-06-01 00,8179.056,17.6,2.5,92.0,0.8,0.0,0,0
1,1,2020-06-01 01,8135.640,17.7,2.9,91.0,0.3,0.0,0,0
2,1,2020-06-01 02,8107.128,17.5,3.2,91.0,0.0,0.0,0,0
3,1,2020-06-01 03,8048.808,17.1,3.2,91.0,0.0,0.0,0,0
4,1,2020-06-01 04,8043.624,17.0,3.3,92.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...
122395,60,2020-08-24 19,4114.368,27.8,2.3,68.0,0.0,0.7,1,1
122396,60,2020-08-24 20,3975.696,27.3,1.2,71.0,0.0,0.0,1,1
122397,60,2020-08-24 21,3572.208,27.3,1.8,71.0,0.0,0.0,1,1
122398,60,2020-08-24 22,3299.184,27.1,1.8,74.0,0.0,0.0,1,1


In [ ]:
test

,num,date_time,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
0,1,2020-08-25 00,27.8,1.5,74.0,0.0,0.0,0,0
1,1,2020-08-25 01,NaN,NaN,NaN,NaN,NaN,0,0
2,1,2020-08-25 02,NaN,NaN,NaN,NaN,NaN,0,0
3,1,2020-08-25 03,27.3,1.1,78.0,NaN,0.0,0,0
4,1,2020-08-25 04,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...
10075,60,2020-08-31 19,NaN,NaN,NaN,NaN,NaN,1,1
10076,60,2020-08-31 20,NaN,NaN,NaN,NaN,NaN,1,1
10077,60,2020-08-31 21,27.9,4.1,68.0,NaN,0.0,1,1
10078,60,2020-08-31 22,NaN,NaN,NaN,NaN,NaN,1,1


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# 시리즈를 지도학습으로
def series_to_supervised(data, n_in = 1, n_out = 1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = pd.DataFrame(data)
  cols, names = list(), list()
  #input sequence (t-m, ... t-1)
  for i in range(n_in, 0, -1):
    cols.append(df.shift(i))
    names += [(('var%d(t-%d') % (j+1, i)) for j in range(n_vars)]
  #forecast sequence
  for i in range(0, n_out):
    cols.append(df.shift(-i))
    if i >=0:
      names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
  agg = pd.concat(cols, axis=1)
  agg.columns = names

  if dropnan:
    agg.dropna(inplace=True)

  return agg
dataset = pd.read_csv('eletrain.csv', index_col =1)
values=dataset.values
encoder = LabelEncoder()
values[:, 0] = encoder.fit_transform(values[:, 0])
values = values.astype('float32')
scaler = MinMaxScaler(feature_range=(0,6))
scaled = scaler.fit_transform(values)
reframed = series_to_supervised(scaled, 4, 1)
print(reframed.head())

   var1(t-4  var2(t-4  var3(t-4  var4(t-4  var5(t-4  var6(t-4  var7(t-4  \
4       0.0  2.766431  1.547619  0.746269  5.407408  0.058896       0.0   
5       0.0  2.751746  1.571429  0.865672  5.333333  0.022086       0.0   
6       0.0  2.742102  1.523810  0.955224  5.333333  0.000000       0.0   
7       0.0  2.722377  1.428571  0.955224  5.333333  0.000000       0.0   
8       0.0  2.720623  1.404762  0.985075  5.407408  0.000000       0.0   

   var8(t-4  var9(t-4  var1(t-3  ...  var9(t-1  var1(t)   var2(t)   var3(t)  \
4       0.0       0.0       0.0  ...       0.0      0.0  2.720623  1.404762   
5       0.0       0.0       0.0  ...       0.0      0.0  2.709445  1.380953   
6       0.0       0.0       0.0  ...       0.0      0.0  2.698486  1.333333   
7       0.0       0.0       0.0  ...       0.0      0.0  2.712295  1.380953   
8       0.0       0.0       0.0  ...       0.0      0.0  2.712952  1.595238   

    var4(t)   var5(t)   var6(t)  var7(t)  var8(t)  var9(t)  
4  0.985075  

In [ ]:
test=test.interpolate()

,num,기온(°C),풍속(m/s),습도(%),"강수량(mm, 6시간)","일조(hr, 3시간)",비전기냉방설비운영,태양광보유
date_time,,,,,,,,
2020-08-25 00,1,27.800000,1.500000,74.000000,0.0,0.000000,0,0
2020-08-25 01,1,27.633333,1.366667,75.333333,0.0,0.000000,0,0
2020-08-25 02,1,27.466667,1.233333,76.666667,0.0,0.000000,0,0
2020-08-25 03,1,27.300000,1.100000,78.000000,0.0,0.000000,0,0
2020-08-25 04,1,26.900000,1.166667,79.666667,0.0,0.000000,0,0
...,...,...,...,...,...,...,...,...
2020-08-31 19,60,28.633333,3.566667,66.000000,0.0,0.533333,1,1
2020-08-31 20,60,28.266667,3.833333,67.000000,0.0,0.266667,1,1
2020-08-31 21,60,27.900000,4.100000,68.000000,0.0,0.000000,1,1


In [ ]:
input_window =996 #임의의 수
output_window = 24 #168 7일 24시간
window = 12 #window는 12시간 마다는 12시간 마다
num_features = 3 #베이스라인은 feature를 하나만 사용, 나느 3개
num_power = 60
end_=168
lstm_units=32
dropout=0.2
EPOCH=30
BATCH_SIZE=128

In [ ]:
mini=train.iloc[:,2].min()
size=train.iloc[:,2].max()-train.iloc[:,2].min()
train.iloc[:,2]=(train.iloc[:,2]-mini)/size
mini2=train.iloc[:,3].min()
size2=train.iloc[:,3].max()-train.iloc[:,3].min()
train.iloc[:,3]=(train.iloc[:,3]-mini2)/size2
mini3=train.iloc[:,5].min()
size3=train.iloc[:,5].max()-train.iloc[:,5].min()
train.iloc[:,5]=(train.iloc[:,5]-mini3)/size3
train_x=tf.reshape(train.iloc[:,[2,3,5]].values, [num_power, 24*85, num_features])
print(f'train_x.shape:{train_x.shape}')

train_x.shape:(60, 2040, 3)


In [ ]:
train_window_x= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, input_window, num_features)) 
train_window_y= np.zeros(( train_x.shape[0], (train_x.shape[1]-(input_window + output_window))//window, output_window, num_features))

In [ ]:
print(f'train_window_x.shape:{train_window_x.shape}')
print(f'train_window_y.shape:{train_window_y.shape}')

train_window_x.shape:(60, 85, 996, 3)
train_window_y.shape:(60, 85, 24, 3)


In [ ]:
for example in range(train_x.shape[0]):
    for start in range(0, train_x.shape[1]-(input_window+output_window), window):
        end=start+input_window
        train_window_x[example, start//window, :] = train_x[example, start: end               , :]
        train_window_y[example, start//window, :] = train_x[example, end  : end+ output_window, :]

In [ ]:
train_window_x

array([[[[0.46107178, 0.25793651, 0.90123457],
         [0.45862432, 0.26190476, 0.88888889],
         [0.45701703, 0.25396825, 0.88888889],
         ...,
         [0.47571999, 0.5       , 0.49382716],
         [0.47524511, 0.51984127, 0.5308642 ],
         [0.47630446, 0.55555556, 0.55555556]],

        [[0.45599422, 0.47619048, 0.37037037],
         [0.45676133, 0.47619048, 0.28395062],
         [0.45595769, 0.49603175, 0.24691358],
         ...,
         [0.4768524 , 0.44047619, 0.87654321],
         [0.47740034, 0.43650794, 0.87654321],
         [0.47641405, 0.42460317, 0.87654321]],

        [[0.44649662, 0.21825397, 0.67901235],
         [0.44477975, 0.20238095, 0.71604938],
         [0.44353775, 0.19047619, 0.72839506],
         ...,
         [0.4779848 , 0.29761905, 0.95061728],
         [0.47882498, 0.31746032, 0.9382716 ],
         [0.47845968, 0.31746032, 0.95061728]],

        ...,

        [[0.47886151, 0.51984127, 0.54320988],
         [0.47816745, 0.50396825, 0.55555556]

In [ ]:
new_train_x=tf.reshape(train_window_x, [-1, input_window, num_features])
new_train_y=tf.reshape(train_window_y, [-1, output_window,num_features])
print(f'new_train_x.shape:{new_train_x.shape}')
print(f'new_train_y.shape:{new_train_y.shape}')

new_train_x.shape:(5100, 996, 3)
new_train_y.shape:(5100, 24, 3)


In [ ]:
model=Sequential([
LSTM(lstm_units, return_sequences=False, recurrent_dropout=dropout),
Dense(output_window * num_features, kernel_initializer=tf.initializers.zeros()),
Reshape([output_window, num_features])
])

In [ ]:
model.compile(optimizer='rmsprop', loss='mae', metrics=['mae'])
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch % 10 == 0: print('')
        print('.', end='')

#가장 좋은 성능을 낸 val_loss가 적은 model만 남겨 놓았습니다.
save_best_only=tf.keras.callbacks.ModelCheckpoint(filepath="lstm_model.h5", monitor='val_loss', save_best_only=True)


early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

#검증 손실이 10epoch 동안 좋아지지 않으면 학습률을 0.1 배로 재구성하는 명령어입니다.
reduceLR = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

In [ ]:
model.fit(new_train_x, new_train_y, epochs=EPOCH, batch_size=BATCH_SIZE, validation_split = 0.2, verbose=0,
          callbacks=[PrintDot(), early_stop, save_best_only , reduceLR])

model.summary()


..........
..........
..........Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 32)                4608      
                                                                 
 dense_6 (Dense)             (None, 72)                2376      
                                                                 
 reshape_3 (Reshape)         (None, 24, 3)             0         
                                                                 
Total params: 6,984
Trainable params: 6,984
Non-trainable params: 0
_________________________________________________________________


In [ ]:
prediction=np.zeros((num_power, end_, num_features))
new_test_x=train_x

for i in range(end_//output_window):
    start_=i*output_window
    next_=model.predict(new_test_x[ : , -input_window:, :])
    new_test_x = tf.concat([new_test_x, next_], axis=1)
    print(new_test_x.shape)
    prediction[:, start_: start_ + output_window, :]= next_

2/2 [==============================] - 1s 381ms/step
(60, 2064, 3)
2/2 [==============================] - 1s 373ms/step
(60, 2088, 3)
2/2 [==============================] - 1s 342ms/step
(60, 2112, 3)
2/2 [==============================] - 1s 341ms/step
(60, 2136, 3)
2/2 [==============================] - 1s 381ms/step
(60, 2160, 3)
2/2 [==============================] - 1s 382ms/step
(60, 2184, 3)
2/2 [==============================] - 1s 342ms/step
(60, 2208, 3)


In [ ]:
prediction[:,:,0]

array([[ 0.42315699,  0.42222505,  0.42240301, ...,  0.23423647,
         0.23626873,  0.23545445],
       [ 0.0339053 ,  0.03209497,  0.03030865, ...,  0.00583371,
        -0.00330224, -0.00484338],
       [ 0.12109302,  0.11861454,  0.11687952, ...,  0.0192478 ,
         0.00921979,  0.00752722],
       ...,
       [ 0.02601471,  0.02373978,  0.02223342, ...,  0.00799893,
        -0.00155407, -0.00312707],
       [ 0.00982087,  0.00654218,  0.00411151, ...,  0.00892523,
        -0.00171698, -0.00333831],
       [ 0.14013135,  0.13830035,  0.13699139, ...,  0.01773785,
         0.00811394,  0.00648973]])

In [ ]:
train = pd.read_csv('eletrain.csv')
mini=train.iloc[:,2].min()
size=train.iloc[:,2].max()-train.iloc[:,2].min()

In [ ]:
prediction[:,:,0] = prediction[:,:,0]*size + mini

In [ ]:
submission['answer'] = prediction[:,:,0].reshape([-1,1])
submission

,num_date_time,answer
0,1 2020-08-25 00,7506.476982
1,1 2020-08-25 01,7489.945090
2,1 2020-08-25 02,7493.102050
3,1 2020-08-25 03,7449.949334
4,1 2020-08-25 04,7421.932877
...,...,...
10075,60 2020-08-31 19,662.867288
10076,60 2020-08-31 20,505.749704
10077,60 2020-08-31 21,314.655647
10078,60 2020-08-31 22,143.935036


In [ ]:
submission.to_csv('submission.csv', index=False)